In [1]:
import copy
import random

class Q_learning_grid:
    def __init__(self,N :int,goal : str, dead_states : list = []):
        if N<2:
            print("N should be greater than or equal to 2")
            exit(0)
        self.N = N
        self.goal = goal.lower()
        self.dead_states = dead_states
        self.construct_Environment()
        self.construct_rewards_initial_Q()

    def construct_Environment(self):
        #n = int(input("Enter N:"))
        self.environment = {}
        k = 1
        for i in range(self.N):
            for j in range(self.N):
                self.environment[(i,j)] = f"s{k}"
                k+=1
        print(self.environment)
            
    def return_index(self,s : str):
        for k in self.environment:
            if self.environment[k] == s:
                return k

    def action_reward(self,action : int, i_p : int , j_p : int):#returns reward and next state index
        if action==0:
            i_n = i_p-1 #i_new
            if i_n<0:
                return [-100,i_p,j_p]#i_previous
            elif self.environment[(i_n,j_p)] in self.dead_states:
                return [-100,i_p,j_p]
            elif self.environment[(i_n,j_p)] == self.goal:
                return [100,i_n,j_p]
            else:
                return [0,i_n,j_p]
        elif action == 1:
            j_n = j_p+1
            if j_n>=self.N:
                return [-100,i_p,j_p]
            elif self.environment[(i_p,j_n)] in self.dead_states:
                return [-100,i_p,j_p]
            elif self.environment[(i_p,j_n)] == self.goal:
                return [100,i_p,j_n]
            else:
                return [0,i_p,j_n]
        elif action == 2:
            i_n = i_p+1
            if i_n>=self.N:
                return [-100,i_p,j_p]
            elif self.environment[(i_n,j_p)] in self.dead_states:
                return [-100,i_p,j_p]
            elif self.environment[(i_n,j_p)] == self.goal:
                return [100,i_n,j_p]
            else:
                return [0,i_n,j_p]
        elif action==3:
            j_n = j_p-1
            if j_n<0:
                return [-100,i_p,j_p]
            elif self.environment[(i_p,j_n)] in self.dead_states:
                return [-100,i_p,j_p]
            elif self.environment[(i_p,j_n)] == self.goal:
                return [100,i_p,j_n]
            else:
                return [0,i_p,j_n]  

    def construct_rewards_initial_Q(self):

        if self.goal in self.dead_states:
            print("Goal can't be dead")
            return
        else:
            self.rewards = {}
            k = 1
            i = 0
            j = 0 
            for i in range(self.N):
                for j in range(self.N):
                    self.rewards[f"s{k}"] = [
                        self.action_reward(0,i,j)[0],#north
                        self.action_reward(1,i,j)[0],#east
                        self.action_reward(2,i,j)[0],#south
                        self.action_reward(3,i,j)[0]#west   
                    ]
                    k=k+1

        self.Q_table = copy.deepcopy(self.rewards)
        print(self.rewards)

    # def add_dead_states(self , d_st : list):
    #     if self.goal in d_st:
    #         print("Goal can't be dead")
    #     elif len(d_st)==0:
    #         return
    #     else:
    #         for i in range(len(d_st)):
    #             self.rewards[d_st[i]] = -100
    #         self.Q_table = copy.deepcopy(self.rewards)
    #         print(self.rewards)

    def make_learn(self,alpha=1.0,gamma=0.9,epochs = 20):
        self.alpha = alpha
        self.gamma = gamma
        start = "s1"
        if (start == self.goal):
            start = "s2"
        i = 0
        for i in range(20):
            state = start
            while state != self.goal:
                action = random.randint(0,3)
                rtplus1 = self.rewards[state][action]
                old_Q = self.Q_table[state][action]

                (a,b) = self.return_index(state)
                after_action = self.action_reward(action,a,b)
                if after_action[0] == -100:
                    next_maxQ = 0
                    self.Q_table[state][action] = rtplus1
                    continue
                next_state = self.environment[(after_action[1],after_action[2])]
                # print(f"{state}-->{next_state}")
                next_maxQ = max(self.Q_table[next_state])
                # print(next_maxQ)
                new_Q = ((1.0-self.alpha)*old_Q )+ (self.alpha*(rtplus1+(self.gamma*next_maxQ)))
                self.Q_table[state][action] = new_Q
                state = next_state
                # print(rewards)
            # print(self.Q_table)
        print(self.Q_table)
        
    def create_policy(self):
        policy = {}
        # print(Q_table)
        for state in self.Q_table:
            policy[state] = self.Q_table[state].index(max(self.Q_table[state]))
        print(policy)
        return policy
    
    def construct_path(self,start):
        start = start.lower()
        path = ""
        policy = self.create_policy()
        state = start 
        path = path+f"{state}"
        while state!=self.goal:
            action = policy[state]
            a,b = self.return_index(state)
            next_state_index = self.action_reward(action,a,b)
            next_state = self.environment[(next_state_index[1],next_state_index[2])]
            state = next_state
            path = path+f"-->{state}"
        
        return(path)

In [ ]:
#picture grid 19x19 (new_grid3.jpeg)
new_grid3 = Q_learning_grid(19,"s73",["s12", "s20", "s21", "s22", "s23", "s25", "s26", "s27", "s29", "s30", "s31", "s33", "s34", "s35", "s36", "s37", "s44", "s48", "s52", "s56", "s59", "s61", "s62", "s63", "s65", "s67", "s68", "s69", "s71", "s75", "s78", "s80", "s84", "s90", "s94", "s96", "s97", "s99", "s100", "s101", "s102", "s103", "s105", "s106", "s107", "s108", "s109", "s110", "s111", "s113", "s118", "s122", "s124", "s130","s135", "s136", "s137", "s139", "s141", "s142", "s143", "s145", "s147", "s148", "s149", "s150", "s151", "s152", "s158", "s164", "s172", "s173", "s174", "s175", "s176", "s177", "s178", "s179", "s181", "s182", "s183", "s184", "s185", "s187", "s188", "s189", "s190","s196", "s204","s211","s212","s213", "s215","s217", "s218", "s219", "s220", "s221", "s223", "s225", "s226", "s227", "s232", "s234", "s236","s240", "s242", "s244","s248","s249", "s251", "s253", "s255","s257", "s259", "s261", "s262", "s263", "s265", "s266", "s270", "s272", "s276", "s278", "s282","s287","s288", "s289", "s291","s292","s293","s294", "s295", "s297","s298","s299", "s301", "s302", "s303", "s306", "s310","s314", "s316", "s320", "s325","s327","s328", "s329","s331", "s333", "s335", "s337", "s338", "s339", "s341", "s342", "s344", "s350", "s354"])
# new_grid2.add_dead_states(["s4","s6","s7","s15","s17","s18","s19"])
new_grid3.make_learn(epochs=15)
path3 = new_grid3.construct_path("s343")
print(path3)

{(0, 0): 's1', (0, 1): 's2', (0, 2): 's3', (0, 3): 's4', (0, 4): 's5', (0, 5): 's6', (0, 6): 's7', (0, 7): 's8', (0, 8): 's9', (0, 9): 's10', (0, 10): 's11', (0, 11): 's12', (0, 12): 's13', (0, 13): 's14', (0, 14): 's15', (0, 15): 's16', (0, 16): 's17', (0, 17): 's18', (0, 18): 's19', (1, 0): 's20', (1, 1): 's21', (1, 2): 's22', (1, 3): 's23', (1, 4): 's24', (1, 5): 's25', (1, 6): 's26', (1, 7): 's27', (1, 8): 's28', (1, 9): 's29', (1, 10): 's30', (1, 11): 's31', (1, 12): 's32', (1, 13): 's33', (1, 14): 's34', (1, 15): 's35', (1, 16): 's36', (1, 17): 's37', (1, 18): 's38', (2, 0): 's39', (2, 1): 's40', (2, 2): 's41', (2, 3): 's42', (2, 4): 's43', (2, 5): 's44', (2, 6): 's45', (2, 7): 's46', (2, 8): 's47', (2, 9): 's48', (2, 10): 's49', (2, 11): 's50', (2, 12): 's51', (2, 13): 's52', (2, 14): 's53', (2, 15): 's54', (2, 16): 's55', (2, 17): 's56', (2, 18): 's57', (3, 0): 's58', (3, 1): 's59', (3, 2): 's60', (3, 3): 's61', (3, 4): 's62', (3, 5): 's63', (3, 6): 's64', (3, 7): 's65', (3, 8)

In [ ]:
#picture grid (new_grid4.jpeg)

In [13]:
new_grid2 = Q_learning_grid(5,"s25",["s4","s6","s7","s15","s17","s18","s19"])
# new_grid2.add_dead_states(["s4","s6","s7","s15","s17","s18","s19"])
new_grid2.make_learn()
path2 = new_grid2.construct_path("s1")
print(path2)

{(0, 0): 's1', (0, 1): 's2', (0, 2): 's3', (0, 3): 's4', (0, 4): 's5', (1, 0): 's6', (1, 1): 's7', (1, 2): 's8', (1, 3): 's9', (1, 4): 's10', (2, 0): 's11', (2, 1): 's12', (2, 2): 's13', (2, 3): 's14', (2, 4): 's15', (3, 0): 's16', (3, 1): 's17', (3, 2): 's18', (3, 3): 's19', (3, 4): 's20', (4, 0): 's21', (4, 1): 's22', (4, 2): 's23', (4, 3): 's24', (4, 4): 's25'}
{'s1': [-100, 0, -100, -100], 's2': [-100, 0, -100, 0], 's3': [-100, -100, 0, 0], 's4': [-100, 0, 0, 0], 's5': [-100, -100, 0, -100], 's6': [0, -100, 0, -100], 's7': [0, 0, 0, -100], 's8': [0, 0, 0, -100], 's9': [-100, 0, 0, 0], 's10': [0, -100, -100, 0], 's11': [-100, 0, 0, -100], 's12': [-100, 0, -100, 0], 's13': [0, 0, -100, 0], 's14': [0, -100, -100, 0], 's15': [0, -100, 0, 0], 's16': [0, -100, 0, -100], 's17': [0, -100, 0, 0], 's18': [0, -100, 0, -100], 's19': [0, 0, 0, -100], 's20': [-100, -100, 100, -100], 's21': [0, 0, -100, -100], 's22': [-100, 0, -100, 0], 's23': [-100, 0, -100, 0], 's24': [-100, 100, -100, 0], 's25

In [2]:
new_grid = Q_learning_grid(4,"s4")
new_grid.make_learn()
path = new_grid.construct_path("s16")
print(path)


{(0, 0): 's1', (0, 1): 's2', (0, 2): 's3', (0, 3): 's4', (1, 0): 's5', (1, 1): 's6', (1, 2): 's7', (1, 3): 's8', (2, 0): 's9', (2, 1): 's10', (2, 2): 's11', (2, 3): 's12', (3, 0): 's13', (3, 1): 's14', (3, 2): 's15', (3, 3): 's16'}
{'s1': [-100, 0, 0, -100], 's2': [-100, 0, 0, 0], 's3': [-100, 100, 0, 0], 's4': [-100, -100, 0, 0], 's5': [0, 0, 0, -100], 's6': [0, 0, 0, 0], 's7': [0, 0, 0, 0], 's8': [100, -100, 0, 0], 's9': [0, 0, 0, -100], 's10': [0, 0, 0, 0], 's11': [0, 0, 0, 0], 's12': [0, -100, 0, 0], 's13': [0, 0, -100, -100], 's14': [0, 0, -100, 0], 's15': [0, 0, -100, 0], 's16': [0, -100, -100, 0]}
{'s1': [-100, 81.0, 65.61000000000001, -100], 's2': [-100, 90.0, 72.9, 72.9], 's3': [-100, 100.0, 81.0, 81.0], 's4': [-100, -100, 0, 0], 's5': [72.9, 72.9, 59.049000000000014, -100], 's6': [81.0, 81.0, 65.61000000000001, 65.61000000000001], 's7': [90.0, 90.0, 72.9, 72.9], 's8': [100.0, -100, 81.0, 81.0], 's9': [65.61000000000001, 65.61000000000001, 53.144100000000016, -100], 's10': [72

In [4]:
# n = int(input("Enter N:"))
# # n= 3
# environment = {}
# k = 1
# for i in range(n):
#     for j in range(n):
#         environment[(i,j)] = f"s{k}"
#         k+=1

# print(environment)



In [5]:
# def return_index(s : str):
#     for k in environment:
#         if environment[k] == s:
#             return k

In [6]:
# goal = input("Enter Goal state:")
# start = "s1"
# # start = "s7"
# # goal = "s3"
# if (start == goal):
#     start = "s2"

# def action_reward(action : int, i_p : int , j_p : int):#returns reward and next state index
#     if action==0:
#         i_n = i_p-1 #i_new
#         if i_n<0:
#             return [-100,i_p,j_p]
#         elif environment[(i_n,j_p)] == goal:
#             return [100,i_n,j_p]
#         else:
#             return [0,i_n,j_p]
#     elif action == 1:
#         j_n = j_p+1
#         if j_n>=n:
#             return [-100,i_p,j_p]
#         elif environment[(i_p,j_n)] == goal:
#             return [100,i_p,j_n]
#         else:
#             return [0,i_p,j_n]
#     elif action == 2:
#         i_n = i_p+1
#         if i_n>=n:
#             return [-100,i_p,j_p]
#         elif environment[(i_n,j_p)] == goal:
#             return [100,i_n,j_p]
#         else:
#             return [0,i_n,j_p]
#     elif action==3:
#         j_n = j_p-1
#         if j_n<0:
#             return [-100,i_p,j_p]
#         elif environment[(i_p,j_n)] == goal:
#             return [100,i_p,j_n]
#         else:
#             return [0,i_p,j_n]
    
# rewards = {}
# k = 1
# i = 0
# j = 0 
# for i in range(n):
#     for j in range(n):
#         rewards[f"s{k}"] = [
#             action_reward(0,i,j)[0],#north
#             action_reward(1,i,j)[0],#east
#             action_reward(2,i,j)[0],#south
#             action_reward(3,i,j)[0]#west   
#         ]
#         k=k+1

# import copy
# Q_table = copy.deepcopy(rewards)
# print(rewards)

In [7]:
# import random
# alpha = 1.0
# gamma = 0.9

# i = 0
# for i in range(20):
#     state = start
#     while state != goal:
#         action = random.randint(0,3)
#         rtplus1 = rewards[state][action]
#         old_Q = Q_table[state][action]

#         (a,b) = return_index(state)
#         after_action = action_reward(action,a,b)
#         if after_action[0] == -100:
#             next_maxQ = 0
#             Q_table[state][action] = rtplus1
#             continue
#         next_state = environment[(after_action[1],after_action[2])]
#         # print(f"{state}-->{next_state}")
#         next_maxQ = max(Q_table[next_state])
#         # print(next_maxQ)
#         new_Q = ((1.0-alpha)*old_Q )+ (alpha*(rtplus1+(gamma*next_maxQ)))
#         Q_table[state][action] = new_Q
#         state = next_state
#         # print(rewards)
#     print(Q_table)


        

In [8]:
# def create_policy():
#     policy = {}
#     # print(Q_table)
#     for state in Q_table:
#         policy[state] = Q_table[state].index(max(Q_table[state]))
#     print(policy)
#     return policy
    
# def construct_path(start,goal):
#     path = ""
#     policy = create_policy()
#     state = start 
#     path = path+f"{state}"
#     while state!=goal:
#         action = policy[state]
#         a,b = return_index(state)
#         next_state_index = action_reward(action,a,b)
#         next_state = environment[(next_state_index[1],next_state_index[2])]
#         state = next_state
#         path = path+f"-->{state}"
#     return(path)

# start = input("Enter start state:")
# path = construct_path(start,goal)

# print(path)

    

exclude the snippets under this

In [9]:

# environment = [
#     ["s1","s2","s3"],
#     ["s4","s5","s6"],
#     ["s7","s8","s9"]
# ]

# def get_index(s: str):
#     for i in range(len(environment)):
#         for j in range(len(environment)):
#             if environment[i][j]==s:
#                 return (i,j)

#stste : [north,east,south,west]
# rewards = {
#     "s1":[0,0,0,0],
#     "s2":[0,100,0,0],
#     "s3":[0,0,0,0],
# }
# "S7".lower()